In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import netCDF4 as ncdf
import matplotlib as mlab
import matplotlib.pyplot as plt
import xarray as xr
import glob as glob
import os as os
import sys as sys
from file_tools import *

# These are 45 vertical levels that the FV3 puts out - use them here to map ARW to that grid for comparison

plevels = np.asarray([100000.,  97500.,  95000.,  92500.,  90000.,  87500.,  85000.,  82500.,
                       80000.,  77500.,  75000.,  72500.,  70000.,  67500.,  65000.,  62500.,
                       60000.,  57500.,  55000.,  52500.,  50000.,  47500.,  45000.,  42500.,
                       40000.,  37500.,  35000.,  32500.,  30000.,  27500.,  25000.,  22500.,
                       20000.,  17500.,  15000.,  12500.,  10000.,   7000.,   5000.,   3000.,
                        2000.,   1000.,    700.,    500.,    200.])

nz_new = plevels.shape[0]

_w_threshold = 5.0
_Rgas       = 287.04
_gravity    = 9.806

# default setting for domain cutting

sw_corner = None
ne_corner = None


In [4]:
#-------------------------------------------------------------------------------------------------------
# Set up file lists for processing
#-------------------------------------------------------------------------------------------------------

date = '2021090100'

# HRRR files

# hrrr_dir   = '/Users/Louis.Wicker/CAM_Case_Studies/20210526/HRRR'
# hrrr_files = sorted(glob.glob(hrrr_dir+'/hrrr.*.grib2'))
hrrr_dir = '/work/larissa.reames/%s/hrrr'%date
hrrr_files = sorted(glob.glob(hrrr_dir+'/*.grib2'))

# RRFS1 dir

#rrfs_dir   = '/Users/Louis.Wicker/CAM_Case_Studies/20210526/RRFS/ctrl'
#rrfs_files = sorted(glob.glob(rrfs_dir+'/PRSLEV.Grb*'))
rrfs_dir1 = '/work/larissa.reames/%s/ctrl'%date
rrfs_files1 = sorted(glob.glob(rrfs_dir1+'/*.grib2'))

# RRFS2 dir

#rrfs_dir   = '/Users/Louis.Wicker/CAM_Case_Studies/20210526/RRFS/ctrl'
#rrfs_files = sorted(glob.glob(rrfs_dir+'/PRSLEV.Grb*'))
rrfs_dir2 = '/work/larissa.reames/%s/nord3'%date
rrfs_files2 = sorted(glob.glob(rrfs_dir2+'/*.grib2'))



In [5]:
#-------------------------------------------------------------------------------------------------------
# Specify regional domain
# This is tricky, you need to know where to put the domain before you can actually see it easily

# One way is to process a single file perhaps when convection is active, and then choose the domain.  
# Ideally, the domains should be nx > 500, ny > 500, e.g., about 12-15 degrees in both dlat and dlon
#-------------------------------------------------------------------------------------------------------
#

if (date == '20210526'):
# May 26 Case (NW KS) domain
    sw_corner = (30.0, -105.0)
    ne_corner = (43.0, -91.0)

# 19 July 2019 (N Plains) domain
elif (date == '2019071918'):
    sw_corner = (38.0, -97.0)
    ne_corner = (50.0, -82.0)
    
# 02 March 2020 (Upper MS Valley) domain
elif (date == '2020030212'):
    sw_corner = (30.0, -93.0)
    ne_corner = (42.0, -80.0)
    
# 03 May 2020 (Central MS Valley) domain
elif (date == '2020050300'):
    sw_corner = (31.0, -96.0)
    ne_corner = (44.0, -81.0)
    
# 07 July 2020 (Central Plains) domain
elif (date == '2020070700'):
    sw_corner = (38.0, -105.0)
    ne_corner = (50.0, -92.0)
    
# 07 July 2020 (Upper MW) domain
elif (date == '2020081006'):
    sw_corner = (35.0, -100.0)
    ne_corner = (48.0, -86.0)

# 01 September 2021 (Mid-Atlantic) domain
elif (date == '2021090100'):
    sw_corner = (32, -82.0)
    ne_corner = (44.0, -69.0)

#-------------------------------------------------------------------------------------------------------
# XXX Case


In [7]:
#-------------------------------------------------------------------------------------------------------
#
# Process the HRRR files - the call to read the grib files returns the xarray data set and the NCDF filename
#
#-------------------------------------------------------------------------------------------------------

for file in hrrr_files:
    hrrr_ds, filename = hrrr_grib_read_variable(file, sw_corner = sw_corner, ne_corner = ne_corner, writeout=True, prefix=None, interpP=True)

------------------------------------------------------------------------------------------------------------------------ 

HRRR Grib READ: Extracting variables from grib file: /work/larissa.reames/2021090100/hrrr/hrrr.t00z.wrfnatf18.grib2 

InterpP is True, Read 3D pressure field from GRIB file

P-max:  102258.83  P-min:  1731.48

Reading my variable:  TEMP from GRIB file variable: Temperature

Reading my variable:  OMEGA from GRIB file variable: Vertical velocity

Reading my variable:  U from GRIB file variable: U component of wind

Reading my variable:  V from GRIB file variable: V component of wind

Reading my variable:  GPH from GRIB file variable: Geopotential Height

Reading my variable:  UH from GRIB file variable: unknown

Reading my variable:  CREF from GRIB file variable: Maximum/Composite radar reflectivity

Reading my variable:  HGT from GRIB file variable: Orography

Creating a sub-region of grid: 32.00, -82.00, 44.00, -69.00 

Successfully wrote new data to file:: /work/l

KeyboardInterrupt: 

In [6]:
#-------------------------------------------------------------------------------------------------------
#
# Process the RRFS files - the call to read the grib files returns the xarray data set and the NCDF filename
#
#-------------------------------------------------------------------------------------------------------

for file in rrfs_files1:
    rrfs_ds1, filename = fv3_grib_read_variable(file, sw_corner = sw_corner, ne_corner = ne_corner, writeout=True, prefix=None)

------------------------------------------------------------------------------------------------------------------------ 

RRFS Grib READ: Extracting variables from grib file: /work/larissa.reames/2021090100/ctrl/PRSLEV.GrbF18.grib2 

Reading my variable:  TEMP from GRIB variable: Temperature

k 0 ind 44 Level 1000 obj_level 1000:    Min 280.2      Max 316.6
k 1 ind 43 Level 975 obj_level 975:    Min 278.9      Max 315.1
k 2 ind 42 Level 950 obj_level 950:    Min 277.6      Max 313.5
k 3 ind 41 Level 925 obj_level 925:    Min 276.5      Max 311.9
k 4 ind 40 Level 900 obj_level 900:    Min 275.5      Max 310.3
k 5 ind 39 Level 875 obj_level 875:    Min 274.4      Max 308.7
k 6 ind 38 Level 850 obj_level 850:    Min 273.2      Max 307.3
k 7 ind 37 Level 825 obj_level 825:    Min 272.6      Max 304.9
k 8 ind 36 Level 800 obj_level 800:    Min 272.2      Max 301.7
k 9 ind 35 Level 775 obj_level 775:    Min 271.0      Max 299.2
k 10 ind 34 Level 750 obj_level 750:    Min 269.9      Max 296.

KeyboardInterrupt: 

In [6]:
#-------------------------------------------------------------------------------------------------------
#
# Process the RRFS files - the call to read the grib files returns the xarray data set and the NCDF filename
#
#-------------------------------------------------------------------------------------------------------

for file in rrfs_files2:
    rrfs_ds2, filename = fv3_grib_read_variable(file, sw_corner = sw_corner, ne_corner = ne_corner, writeout=True, prefix=None)

------------------------------------------------------------------------------------------------------------------------ 

RRFS Grib READ: Extracting variables from grib file: /work/larissa.reames/2021090100/nord3/PRSLEV.GrbF18.grib2 

Reading my variable:  TEMP from GRIB variable: Temperature

k 0 ind 44 Level 1000 obj_level 1000:    Min 280.2      Max 316.5
k 1 ind 43 Level 975 obj_level 975:    Min 278.9      Max 315.0
k 2 ind 42 Level 950 obj_level 950:    Min 277.6      Max 313.4
k 3 ind 41 Level 925 obj_level 925:    Min 276.5      Max 311.9
k 4 ind 40 Level 900 obj_level 900:    Min 275.5      Max 310.2
k 5 ind 39 Level 875 obj_level 875:    Min 274.4      Max 308.6
k 6 ind 38 Level 850 obj_level 850:    Min 273.2      Max 307.5
k 7 ind 37 Level 825 obj_level 825:    Min 272.5      Max 304.8
k 8 ind 36 Level 800 obj_level 800:    Min 272.2      Max 301.6
k 9 ind 35 Level 775 obj_level 775:    Min 271.0      Max 299.1
k 10 ind 34 Level 750 obj_level 750:    Min 269.9      Max 296